Install QDTrack
a. Create a conda virtual environment and activate it.

```
conda create -n qdtrack python=3.7 -y
conda activate qdtrack
```

b. Install PyTorch and torchvision following the official instructions, e.g.,

```
conda install pytorch torchvision -c pytorch
```

Note: Make sure that your compilation CUDA version and runtime CUDA version match. You can check the supported CUDA version for precompiled packages on the PyTorch website.

c. Install mmcv and mmdetection.

```
pip install mmcv-full==1.6.1
pip install mmdet==2.25.1
```

You can also refer to the offical instructions.

d. Install QDTrack

```
python setup.py develop
```


Step1. Install ByteTrack.
```shell
git clone https://github.com/ifzhang/ByteTrack.git
cd ByteTrack
pip3 install -r requirements.txt
python3 setup.py develop
```

Step2. Install [pycocotools](https://github.com/cocodataset/cocoapi).

```shell
pip3 install cython; pip3 install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
```

Step3. Others
```shell
pip3 install cython_bbox
```

In [5]:
import os
import numpy as np
from tqdm import tqdm
import pickle
from scipy.spatial import KDTree
from webcolors import (
    CSS3_HEX_TO_NAMES,
    hex_to_rgb,
)

data_dir = "./data/tracker/pickles"


In [6]:
labels_dict = {
    0:  {'name': 'unlabeled',      'color': (0,   0,   0),   'ct_scape_id': 0,  'train_id': 255},
    1:  {'name': 'road',           'color': (128, 64,  128), 'ct_scape_id': 7,  'train_id': 1},
    2:  {'name': 'sidewalk',       'color': (244, 35,  232), 'ct_scape_id': 8,  'train_id': 2},
    3:  {'name': 'building',       'color': (70,  70,  70),  'ct_scape_id': 11, 'train_id': 3},
    4:  {'name': 'wall',           'color': (102, 102, 156), 'ct_scape_id': 12, 'train_id': 4},
    5:  {'name': 'fence',          'color': (190, 153, 153), 'ct_scape_id': 13, 'train_id': 5},
    6:  {'name': 'pole',           'color': (153, 153, 153), 'ct_scape_id': 17, 'train_id': 6},
    7:  {'name': 'traffic light',  'color': (250, 170, 30),  'ct_scape_id': 19, 'train_id': 7},
    8:  {'name': 'traffic sign',   'color': (220, 220, 0),   'ct_scape_id': 20, 'train_id': 8},
    9:  {'name': 'vegetation',     'color': (107, 142, 35),  'ct_scape_id': 21, 'train_id': 9},
    10: {'name': 'terrain',        'color': (152, 251, 152), 'ct_scape_id': 22, 'train_id': 10},
    11: {'name': 'sky',            'color': (70,  130, 180), 'ct_scape_id': 23, 'train_id': 11},
    12: {'name': 'person',         'color': (220, 20,  60),  'ct_scape_id': 24, 'train_id': 12},
    13: {'name': 'rider',          'color': (255, 0,   0),   'ct_scape_id': 25, 'train_id': 13},
    14: {'name': 'car',            'color': (0,   0,   142), 'ct_scape_id': 26, 'train_id': 14},
    15: {'name': 'truck',          'color': (0,   0,   70),  'ct_scape_id': 27, 'train_id': 15},
    16: {'name': 'bus',            'color': (0,   60,  100), 'ct_scape_id': 28, 'train_id': 16},
    17: {'name': 'train',          'color': (0,   80,  100), 'ct_scape_id': 31, 'train_id': 17},
    18: {'name': 'motorcycle',     'color': (0,   0,   230), 'ct_scape_id': 32, 'train_id': 18},
    19: {'name': 'bicycle',        'color': (119, 11,  32),  'ct_scape_id': 33, 'train_id': 19},
    20: {'name': 'dynamic',        'color': (111, 74,  0),   'ct_scape_id': 5,  'train_id': 20},
    21: {'name': 'ground',         'color': (81,  0,   81),  'ct_scape_id': 6,  'train_id': 21},
    22: {'name': 'parking',        'color': (250, 170, 160), 'ct_scape_id': 9,  'train_id': 22},
    23: {'name': 'rail track',     'color': (230, 150, 140), 'ct_scape_id': 10, 'train_id': 23},
    24: {'name': 'guard rail',     'color': (180, 165, 180), 'ct_scape_id': 14, 'train_id': 24},
    25: {'name': 'bridge',         'color': (150, 100, 100), 'ct_scape_id': 15, 'train_id': 25},
    26: {'name': 'tunnel',         'color': (150, 120, 90),  'ct_scape_id': 16, 'train_id': 26},
    27: {'name': 'polegroup',      'color': (153, 153, 153), 'ct_scape_id': 18, 'train_id': 27},
    28: {'name': 'caravan',        'color': (0,   0,   90),  'ct_scape_id': 29, 'train_id': 28},
    29: {'name': 'trailer',        'color': (0,   0,   110), 'ct_scape_id': 30, 'train_id': 29}
}


In [7]:
all_pickles = {}

for key in tqdm(labels_dict.keys()):
    with open(os.path.join(data_dir, f'trackers_{labels_dict[key]["name"]}.pickle'), 'rb') as f:
        data = pickle.load(f)
    all_pickles[labels_dict[key]["name"]] = data
# all_pickles

100%|██████████| 30/30 [00:00<00:00, 334.83it/s]


In [8]:
all_pickles

{'unlabeled': {},
 'road': {0: {'01501': {'b_box': [465, 605, 508, 651],
    'color': array([26, 27, 30], dtype=uint8)},
   '01502': {'b_box': [460, 600, 513, 656],
    'color': array([26, 27, 29], dtype=uint8)},
   '01503': {'b_box': [455, 595, 516, 660],
    'color': array([26, 27, 30], dtype=uint8)},
   '01504': {'b_box': [451, 590, 516, 664],
    'color': array([27, 28, 31], dtype=uint8)},
   '01505': {'b_box': [447, 586, 520, 668],
    'color': array([27, 28, 32], dtype=uint8)},
   '01506': {'b_box': [443, 582, 533, 672],
    'color': array([29, 30, 33], dtype=uint8)},
   '01507': {'b_box': [504, 591, 559, 647],
    'color': array([40, 41, 43], dtype=uint8)},
   '01508': {'b_box': [513, 590, 574, 648],
    'color': array([41, 42, 45], dtype=uint8)},
   '01509': {'b_box': [521, 588, 580, 652],
    'color': array([45, 47, 49], dtype=uint8)},
   '01510': {'b_box': [528, 586, 582, 656],
    'color': array([48, 50, 52], dtype=uint8)},
   '01511': {'b_box': [534, 584, 581, 661],
    'co